In [6]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic


In [31]:
import cv2

cap = cv2.VideoCapture("C:/Users/chengyang/Physio/videos/video_2023-11-15_22-03-58.mp4")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = 20

fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Use 'mp4v' or 'h264' instead of 'MP4V'
output_file = cv2.VideoWriter('output1-1.avi', fourcc, 20.0, frame_size)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("hi")
        break

    cv2.imshow("Mediapipe Feed", frame)
    output_file.write(frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
output_file.release()
cv2.destroyAllWindows()


hi


# Make detections

In [7]:
# video feed
import time


cap = cv2.VideoCapture(0)
# fourcc = cv2.VideoWriter_fourcc(*'MP4V')
# file = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640,480))

# setup mediapipe instance
with mp_holistic.Holistic(model_complexity = 0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # new_frame_time = time.time() 
        
        # Calculating the fps 
    
        # fps will be number of frame processed in given time frame 
        # since their will be most of time error of 0.001 second 
        # we will be subtracting it to get more accurate result 
        # fps = 1/(new_frame_time-prev_frame_time) 
        # prev_frame_time = new_frame_time 

        # recoloring the image from bgr to rgb, opencv gives a format in bgr but mediapipe needs rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                #   mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                #   mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )
        # cv2.putText(image, 'FPS: ' + str(int(fps)), (225,12), 
        #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        # file.write(image)
        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # file.release()
    cap.release()
    cv2.destroyAllWindows()

# Extract landmarks for joints

![Alt text](image.png)

In [21]:
# video feed
cap = cv2.VideoCapture(0)

# setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # recoloring the image from bgr to rgb, opencv gives a format in bgr but mediapipe needs rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                #   mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                #   mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.56352276
y: 0.58739316
z: -1.2102833
visibility: 0.9999095
, x: 0.5864028
y: 0.5091506
z: -1.1467639
visibility: 0.99984944
, x: 0.6052167
y: 0.51085424
z: -1.1469095
visibility: 0.9997867
, x: 0.62340754
y: 0.51359606
z: -1.1476065
visibility: 0.9998418
, x: 0.5242032
y: 0.5110048
z: -1.1291313
visibility: 0.99982053
, x: 0.50119245
y: 0.51260376
z: -1.1282583
visibility: 0.9997377
, x: 0.4809088
y: 0.51344585
z: -1.1283782
visibility: 0.9998085
, x: 0.6487872
y: 0.547807
z: -0.66233706
visibility: 0.999859
, x: 0.4478944
y: 0.54578346
z: -0.54915184
visibility: 0.99979836
, x: 0.60186106
y: 0.6548462
z: -1.0293653
visibility: 0.9999355
, x: 0.5232864
y: 0.6591904
z: -0.99926645
visibility: 0.99989927
, x: 0.82388955
y: 0.83747196
z: -0.41451246
visibility: 0.9988194
, x: 0.30463177
y: 0.9010566
z: -0.28518096
visibility: 0.99827385
, x: 0.99262875
y: 1.0883889
z: -0.38059708
visibility: 0.4273908
, x: 0.08262092
y: 1.0656056
z: -0.9882704
visibility: 0.9519027
, x: 1.0816859
y:

# Calulate angle

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 



In [3]:
size_of_image = [640, 480]

def calculate_angle_for_left_curl(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    shoulder = [left_shoulder.x, left_shoulder.y]
    elbow = [left_elbow.x, left_elbow.y]
    wrist = [left_wrist.x, left_wrist.y]

    angle = calculate_angle(shoulder, elbow, wrist)

    position_of_text = np.multiply(elbow, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_right_curl(landmarks):
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

    shoulder = [right_shoulder.x, right_shoulder.y]
    elbow = [right_elbow.x, right_elbow.y]
    wrist = [right_wrist.x, right_wrist.y]

    angle = calculate_angle(shoulder, elbow, wrist)

    position_of_text = np.multiply(elbow, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_right_knee(landmarks):
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

    hip = [right_hip.x, right_hip.y]
    knee = [right_knee.x, right_knee.y]
    ankle = [right_ankle.x, right_ankle.y]

    angle = calculate_angle(hip, knee, ankle)

    position_of_text = np.multiply(knee, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_left_knee(landmarks):
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    hip = [left_hip.x, left_hip.y]
    knee = [left_knee.x, left_knee.y]
    ankle = [left_ankle.x, left_ankle.y]

    angle = calculate_angle(hip, knee, ankle)

    position_of_text = np.multiply(knee, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_right_hip(landmarks):
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

    shoulder = [right_shoulder.x, right_shoulder.y]
    hip = [right_hip.x, right_hip.y]
    knee = [right_knee.x, right_knee.y]

    angle = calculate_angle(shoulder, hip, knee)

    position_of_text = np.multiply(hip, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_left_hip(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

    shoulder = [left_shoulder.x, left_shoulder.y]
    hip = [left_hip.x, left_hip.y]
    knee = [left_knee.x, left_knee.y]

    angle = calculate_angle(shoulder, hip, knee)

    position_of_text = np.multiply(hip, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_right_ankle(landmarks):
    right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

    hip = [right_foot_index.x, right_foot_index.y]
    knee = [right_knee.x, right_knee.y]
    ankle = [right_ankle.x, right_ankle.y]

    angle = calculate_angle(hip, knee, ankle)

    position_of_text = np.multiply(ankle, size_of_image).astype(int)

    return angle, position_of_text

def calculate_angle_for_left_ankle(landmarks):
    left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    hip = [left_foot_index.x, left_foot_index.y]
    knee = [left_knee.x, left_knee.y]
    ankle = [left_ankle.x, left_ankle.y]

    angle = calculate_angle(hip, knee, ankle)

    position_of_text = np.multiply(ankle, size_of_image).astype(int)

    return angle, position_of_text

In [9]:

calculate_angle_for_left_curl(landmarks)

(158.09253374499468, array([438, 414]))

In [4]:
import time


def calc_fps(video):
    new_frame_time = time.time() 
    fps = 1/(new_frame_time-prev_frame_time) 
    prev_frame_time = new_frame_time 
    return fps

In [7]:
# video feed
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
file = cv2.VideoWriter('cc.mp4', fourcc, 20.0, frame_size)

counter = 0
stage = None
# setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # recoloring the image from bgr to rgb, opencv gives a format in bgr but mediapipe needs rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            left_curl_angle, left_curl_position_of_text = calculate_angle_for_left_curl(landmarks)
            
            # Visualize angle
            # cv2.putText(image, 
            #             str(left_curl_angle), 
            #             tuple(left_curl_position_of_text), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 
            #             2, 
            #             cv2.LINE_AA
            # )
            rigth_curl_angle, right_curl_position_of_text = calculate_angle_for_right_curl(landmarks)
            
            # # Visualize angle
            # cv2.putText(image, 
            #             str(rigth_curl_angle), 
            #             tuple(right_curl_position_of_text), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 
            #             2, 
            #             cv2.LINE_AA
            # )
            # Curl counter logic
            if rigth_curl_angle > 160:
                stage = "down"
            if rigth_curl_angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                #   mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                #   mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )
        file.write(image)
        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    file.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
7


In [13]:
import cv2
import mediapipe as mp
import time
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

# VID_DIM = (1290, 1080)
counter = 0
good_counter = 0
bad_counter = 0
cap = cv2.VideoCapture(0)
# used to record the time when we processed last frame 
prev_frame_time = 0
  
# used to record the time at which we processed current frame 
new_frame_time = 0
stage = None
feedback = None
list_of_stages = []
angle_of_right_knee = []
angle_of_left_knee = []

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
file = cv2.VideoWriter('output2.mp4', fourcc, 20.0, frame_size)

with mp_holistic.Holistic(
    model_complexity = 0,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.9) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break

    new_frame_time = time.time() 
  
    # Calculating the fps 
  
    # fps will be number of frame processed in given time frame 
    # since their will be most of time error of 0.001 second 
    # we will be subtracting it to get more accurate result 
    fps = 1/(new_frame_time-prev_frame_time) 
    prev_frame_time = new_frame_time 

    image = cv2.flip(image, 1)
    # image = cv2.resize(image, VID_DIM)
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    try:
        landmarks = results.pose_landmarks.landmark
        left_curl_angle, left_curl_position_of_text = calculate_angle_for_left_curl(landmarks)
        rigth_curl_angle, right_curl_position_of_text = calculate_angle_for_right_curl(landmarks)
        
        left_hip_angle, left_hip_position_of_text = calculate_angle_for_left_hip(landmarks)
        right_hip_angle, right_hip_position_of_text = calculate_angle_for_right_hip(landmarks)

        right_knee_angle, right_knee_position_of_text = calculate_angle_for_right_knee(landmarks)
        left_knee_angle, left_knee_position_of_text = calculate_angle_for_left_knee(landmarks)
        angle_of_left_knee.append(left_knee_angle)
        angle_of_right_knee.append(right_knee_angle)

        right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        left_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
        right_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]

        print(stage)
        
        if (right_knee_angle > 160 or left_knee_angle > 160) and list_of_stages == []:
            stage = "UP"
            list_of_stages.append(stage)
        if (right_knee_angle <= 90 or left_knee_angle <= 90) and list_of_stages[-1] =='Transition':
            stage="DOWN"
            list_of_stages.append(stage)
        if "DOWN" in list_of_stages and (right_knee_angle > 160 or left_knee_angle > 160):
            stage = "UP"
            counter += 1
            feedback = None
            print(list_of_stages)
            if "Bad Transition" not in list_of_stages:
                good_counter += 1
            else:
                bad_counter += 1

            list_of_stages = []
            angle_of_left_knee = []
            angle_of_right_knee = []
        if (right_knee_angle <= 160 and right_knee_angle > 90) or (left_knee_angle <= 160 and left_knee_angle > 90):
            stage = "Transition"
            list_of_stages.append(stage)
            
        if stage == "DOWN" and (right_knee_angle < 50 or left_knee_angle < 50):
            stage = "Bad Transition"
            list_of_stages.append(stage)
            feedback = "Squat too low!"

        if "UP" in list_of_stages and stage == "Transition" and "DOWN" not in list_of_stages:
            min_left_knee_angle = min(angle_of_left_knee)
            min_right_knee_angle = min(angle_of_right_knee)
            if (min_left_knee_angle > 90 or min_right_knee_angle > 90) and (min_left_knee_angle < 130 or min_right_knee_angle < 130):
                stage = "Bad Transition"
                list_of_stages.append(stage)
                feedback = "Bend your knees more!"
        # if stage == "DOWN":
        # if (left_hip_angle > 90 or right_hip_angle > 90) and stage == "DOWN":
        #     print("gay", stage)
        #     feedback = "Bring hips lower"
        #     bad_counter += 1
        #     stage = "Transition"
        # # Example: Check if the knees go below the hips (excessive depth)
        # if (left_knee.y < left_hip.y or right_knee.y < right_hip.y) and stage == "DOWN":
        #     feedback = "Hips should not be below knees"
        #     good_counter -= 1
        #     bad_counter += 1
        #     stage = "Transition"
            
        # elif left_hip_angle < 90 and right_hip_angle < 90 and stage != "Good Squat":
        #     feedback = "Good Squat"
        #     good_counter += 1

                    
    except:
        pass
    
    # Render curl counter
    # Setup status box
    cv2.rectangle(image, (0,0), (325,73), (245,117,16), -1)
    
    # Rep data
    cv2.putText(image, 'REPS', (15,20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter), 
                (10,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.putText(image, 'Good REPS', (15,100), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, str(good_counter), 
                (10,160), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.putText(image, 'Bad REPS', (15,200), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, str(bad_counter), 
                (10,260), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    # Stage data
    cv2.putText(image, 'STAGE', (115,20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, stage, 
                (100,60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.putText(image, 'FEEDBACK', (115,200), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, feedback, 
                (190,260), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.putText(image, 'FPS: ' + str(int(fps)), (560,20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

    # Write the frame to the video file
    file.write(image)

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
file.release()
cv2.destroyAllWindows()

None
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP
UP